In [1]:
#!pip install gensim
import gensim
import spacy


In [2]:
from bs4 import BeautifulSoup
import os.path
import glob
import re
from xml.dom import minidom
import nltk
import nltk
import gensim
import random
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import pandas as pd
import random
from gensim.utils import simple_preprocess

%pprint 





[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gonza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Pretty printing has been turned OFF


In [3]:
#import keras
#import keras.preprocessing.sequence as seq

from sklearn.decomposition import PCA
from matplotlib import pyplot
#from keras.layers import Input, Activation
#from keras.models import Model
#from keras.layers.embeddings import Embedding
from gensim import models
from scipy import spatial
from datetime import datetime
from string import digits
from string import punctuation
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
from datetime import datetime


In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])
##stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import data

In [5]:
df1  = pd.read_csv('../data/cambridge_analytica/regional_datasets/english_northamerica_tweets_20190411.csv')


In [6]:
len(df1)

342400

In [7]:
df2  = pd.read_csv('../data/cambridge_analytica/regional_datasets/english_europe_tweets_20190411.csv')


In [8]:
len(df2)

111745

In [9]:
df = pd.concat([df1, df2])

In [10]:
len(df)

454145

In [11]:
len(df) == (len(df1)+len(df2))

True

In [12]:
df['text'] = df['texto_completo']

In [13]:
df.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,1,2018-04-01,id-980480936205783047,Sun Apr 01 16:24:06 +0000 2018,stanscott53,"Coldspring, TX",NaN,<usernameremoved> Hold Facebook accountable fo...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States
1,6,2018-04-01,id-980481012550524928,Sun Apr 01 16:24:24 +0000 2018,RShaffer1,"Boston, MA",NaN,Both sides should just drop it. Trump won...el...,NaN,NaN,NaN,Both sides should just drop it. Trump won...el...,United States
2,9,2018-04-01,id-980480980191404032,Sun Apr 01 16:24:16 +0000 2018,pcampisi14,"Toronto, Ontario",NaN,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...,NaN,NaN,$AAPL lobbing threes while they can. Wasnt it ...,Canada
3,16,2018-04-01,id-980481205492682753,Sun Apr 01 16:25:10 +0000 2018,CoinSignalBot,"Seattle, WA",NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,NaN,NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,United States
4,20,2018-04-01,id-980481164334063616,Sun Apr 01 16:25:00 +0000 2018,kimbee802009,"South Dakota, USA",NaN,<usernameremoved> Hold Facebook accountable fo...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States


# remove links

In [14]:
df['text'].replace({"http\\S+": '<linkremoved>'}, inplace=True, regex=True)
df['text'].replace({"<link removed>": '<linkremoved>'}, inplace=True, regex=True)


In [15]:
for texto in df['text'][0:5]:
    print(texto)


<usernameremoved> Hold Facebook accountable for abusing our data and censoring our voice. Vote no on the CRA. <linkremoved>
Both sides should just drop it. Trump won...elections over. Move on. <linkremoved>
$AAPL lobbing threes while they can. Wasnt it just a few years back when their platform was actually hacked and people private photos were exposed? <linkremoved>
15 min #RSI Signals:

$BTC - $SLR: 6.84
$BTC - $OMNI: 6.97
$BTC - $RBY: 6.99
$BTC - $SPHR: 21.13
$BTC - $RDN: 21.17
$BTC - $MCO: 21.48
$BTC - $VIBE: 23.76
$BTC - $ADX: 26.25
$BTC - $PPT: 26.63

#Bitcoin #tokensale #smartcontract #ETH #DAPP $LTC #signals #RADS #BigData #litecoin
<usernameremoved> Hold Facebook accountable for abusing our data and censoring our voice. Vote no on the CRA.

<linkremoved>


In [16]:
df.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,1,2018-04-01,id-980480936205783047,Sun Apr 01 16:24:06 +0000 2018,stanscott53,"Coldspring, TX",NaN,<usernameremoved> Hold Facebook accountable fo...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States
1,6,2018-04-01,id-980481012550524928,Sun Apr 01 16:24:24 +0000 2018,RShaffer1,"Boston, MA",NaN,Both sides should just drop it. Trump won...el...,NaN,NaN,NaN,Both sides should just drop it. Trump won...el...,United States
2,9,2018-04-01,id-980480980191404032,Sun Apr 01 16:24:16 +0000 2018,pcampisi14,"Toronto, Ontario",NaN,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...,NaN,NaN,$AAPL lobbing threes while they can. Wasnt it ...,Canada
3,16,2018-04-01,id-980481205492682753,Sun Apr 01 16:25:10 +0000 2018,CoinSignalBot,"Seattle, WA",NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,NaN,NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,United States
4,20,2018-04-01,id-980481164334063616,Sun Apr 01 16:25:00 +0000 2018,kimbee802009,"South Dakota, USA",NaN,<usernameremoved> Hold Facebook accountable fo...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States


# remove usernames

In [17]:
df['text'].replace({"@[^\\s]+": '<usernameremoved>'}, inplace=True, regex=True)

# remove duplicates

In [18]:
df.drop_duplicates(['text'],keep='first', inplace=True)

In [19]:
len(df)

454145

In [20]:
# numero de usuarios
tweets_by_user_data = df.groupby('user.screen_name').agg('size').sort_values(ascending=False).reset_index()
len(tweets_by_user_data)

190646

# remove digits, punctuation, symbols

In [21]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
punctuation+='¡¿'

In [23]:
#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 

In [24]:

import unidecode

data = []
for tweet in df['text']:
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    tweet = unidecode.unidecode(tweet)
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    data.append(tweet)


In [25]:
data[0:5]

[' usernameremoved  hold facebook accountable for abusing our data and censoring our voice  vote no on the cra   linkremoved ', 'both sides should just drop it  trump won   elections over  move on   linkremoved ', ' aapl lobbing threes while they can  wasnt it just a few years back when their platform was actually hacked and people private photos were exposed   linkremoved ', '   min  rsi signals \n\n btc    slr      \n btc    omni      \n btc    rby      \n btc    sphr       \n btc    rdn       \n btc    mco       \n btc    vibe       \n btc    adx       \n btc    ppt       \n\n bitcoin  tokensale  smartcontract  eth  dapp  ltc  signals  rads  bigdata  litecoin', ' usernameremoved  hold facebook accountable for abusing our data and censoring our voice  vote no on the cra \n\n linkremoved ']

In [26]:
len(data)

454145

# Twitter tokenizer

In [27]:
from nltk.tokenize import TweetTokenizer

In [28]:
tknzr = TweetTokenizer()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(tknzr.tokenize(sentence))


In [29]:
data_words = list(sent_to_words(data))


# Create bigram and trigram models

In [30]:
# Build the bigram and trigram models
#quizas el min count es muy bajo
#documentaicon de bigramas 
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#3importpackages
min_count = int(len(df)*0.02)
bigram = gensim.models.Phrases(data_words, min_count=min_count, threshold=1) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], min_count =min_count, threshold=1)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

['both', 'sides', 'should', 'just', 'drop', 'it', 'trump', 'won', 'elections', 'over', 'move', 'on', 'linkremoved']


In [31]:
# See trigram example
#print(trigram_mod[bigram_mod[data_words[1]]])
for i in range(0,5):
    print(trigram_mod[bigram_mod[data_words[i]]])

['usernameremoved', 'hold', 'facebook', 'accountable', 'for', 'abusing', 'our', 'data', 'and', 'censoring', 'our', 'voice', 'vote', 'no', 'on', 'the', 'cra', 'linkremoved']
['both', 'sides', 'should', 'just', 'drop', 'it', 'trump', 'won', 'elections', 'over', 'move', 'on', 'linkremoved']
['aapl', 'lobbing', 'threes', 'while', 'they', 'can', 'wasnt', 'it', 'just', 'a', 'few', 'years', 'back', 'when', 'their', 'platform', 'was', 'actually', 'hacked', 'and', 'people', 'private', 'photos', 'were', 'exposed', 'linkremoved']
['min', 'rsi', 'signals', 'btc', 'slr', 'btc', 'omni', 'btc', 'rby', 'btc', 'sphr', 'btc', 'rdn', 'btc', 'mco', 'btc', 'vibe', 'btc', 'adx', 'btc', 'ppt', 'bitcoin', 'tokensale', 'smartcontract', 'eth', 'dapp', 'ltc', 'signals', 'rads', 'bigdata', 'litecoin']
['usernameremoved', 'hold', 'facebook', 'accountable', 'for', 'abusing', 'our', 'data', 'and', 'censoring', 'our', 'voice', 'vote', 'no', 'on', 'the', 'cra', 'linkremoved']


# Remove stopwords, make bigrams and lemmatize

In [32]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [33]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [34]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


In [35]:
data_words_nostops[0]

['hold', 'facebook', 'accountable', 'abusing', 'data', 'censoring', 'voice', 'vote', 'cra']

In [36]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)



In [37]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
!python -m spacy download en

C:\Users\gonza\tesisenv\Scripts\python.exe: No module named spacy


In [38]:

nlp = spacy.load('en', disable=['parser', 'ner'])

In [39]:

#sera mejor ocupar wordnetlemattizer?
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote']]


In [40]:
data_lemmatized[0]

['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote']

In [41]:
#texts should be a list of list, where each sublist is a lemmatized sentence
texts = data_lemmatized



In [42]:
len(texts)

454145

In [43]:
texts[0:10]

[['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote'], ['side', 'drop', 'trump', 'election', 'move'], ['lob', 'three', 'year', 'back', 'platform', 'actually', 'hack', 'people', 'private', 'photo', 'expose'], ['signal', 'rad', 'bigdata'], ['hold', 'accountable', 'abusing', 'datum', 'censor', 'voice', 'vote'], ['lmfao'], ['user', 'lock', 'horse', 'go'], ['worried', 'ethic', 'lol'], ['ship', 'even', 'trust', 'much', 'get', 'tired'], ['sell', 'dietary', 'habit', 'know', 'eat', 'frozen']]

# vocabulario

In [44]:
conjunto_palabras = set()
for sentence in texts:
    for term in sentence:
        conjunto_palabras.add(term)
print("There are:"+str(len(conjunto_palabras)))

There are:67706


In [45]:
list_terms_that_may_not_appear = ['deletefacebook', 'retweete', 'gdpr', 'socialmedia', 'bigdata','machinelearne', 'datascience', 'deeplearne', 'retweeted']
for term in list_terms_that_may_not_appear:
    print(term, "in set:", term in conjunto_palabras)

deletefacebook in set: True
retweete in set: True
gdpr in set: True
socialmedia in set: True
bigdata in set: True
machinelearne in set: True
datascience in set: True
deeplearne in set: True
retweeted in set: True


# crear word embeddings - best architecture so far according to EPJ data science

In [46]:
print("Total number of tweets: "+str(len(texts)))

Total number of tweets: 454145


In [47]:
#https://colab.research.google.com/drive/1c0KFI1FEFNnfmKWrBFpR2GGI76_d0rg0?authuser=1#scrollTo=Y_CF5UpqPXuf
start_time = datetime.now()
model_created = Word2Vec(texts, size=300, iter = 50, sg=0, window=5, workers=4, negative =5, compute_loss=True, min_count=3) #min_count permite hacer un trim
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
#https://docs.google.com/spreadsheets/d/1qhTmk3xxMW-NGLO2ZlvwSYzv7sI5i3RwlsIZAaY5Ndc/edit#gid=1606388650
model_created.save('embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model')
model_created.wv.save_word2vec_format('embedding_english__europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.bin',binary=True)

Duration: 0:02:26.684719


In [ ]:
#https://docs.google.com/spreadsheets/d/1F0HHOTjnQxmV2vQjMrKo_9RXoeoF8Nw59R1vXsX5tVM/edit#gid=0

In [50]:
list_terms_that_may_not_appear = ['deletefacebook', 'retweete', 'gdpr', 'socialmedia', 'bigdata','machinelearne', 'datascience', 'deeplearne', 'retweeted']
for term in list_terms_that_may_not_appear:
    print(term, "in set:", term in model_created.wv)

deletefacebook in set: True
retweete in set: True
gdpr in set: True
socialmedia in set: True
bigdata in set: True
machinelearne in set: True
datascience in set: True
deeplearne in set: True
retweeted in set: True
